In [145]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [146]:
df = pd.read_csv("test.csv")

In [147]:
df.head()

,trip_ID,visitor_nation,age_bracket,travelling_with,female_count,male_count,key_activity,trip_purpose,first_time_visitor,mainland_nights,...,package_accomodation,food_package,transport_package_mx,sightseeing_package,guided_tour_package,insurance_package,days_before_booked,weather_at_arrival,tour_length,special_requirements
0,tour_id8gzpck76,CONGO,25-44,Alone,0.0,1.0,Widlife Tourism,Business,No,14,...,No,No,No,No,No,No,15-30,"sunny,",15-30,"wheelchair,"
1,tour_idow1zxkou,SWIZERLAND,45-64,With Spouse,1.0,1.0,Widlife Tourism,Leisure and Holidays,Yes,8,...,Yes,Yes,No,No,Yes,No,61-90,Stormy,1-6,NaN
2,tour_idue7esfqz,MEXICO,45-64,With Other Friends/Relatives,2.0,0.0,Cultural Tourism,Leisure and Holidays,Yes,3,...,Yes,Yes,Yes,Yes,Yes,No,1-7,Stormy,30+,none
3,tour_idnj3mjzpb,JAPAN,25-44,With Other Friends/Relatives,1.0,1.0,Widlife Tourism,Leisure and Holidays,Yes,5,...,Yes,Yes,Yes,Yes,Yes,No,90+,"sunny,",7-14,NaN
4,tour_ida3us5yk2,SPAIN,25-44,With Other Friends/Relatives,2.0,0.0,Wildlife Tourism,Leisure and Holidays,Yes,0,...,Yes,Yes,No,Yes,No,Yes,61-90,Stormy,30+,"dietary needs,"


In [148]:
df.isna().sum()

trip_ID                               0
visitor_nation                      126
age_bracket                           0
travelling_with                     338
female_count                          0
male_count                            2
key_activity                         65
trip_purpose                          0
first_time_visitor                   39
mainland_nights                       0
island_nights                         0
tour_arrangement                      0
transport_package_international      57
source_of_info                        0
package_accomodation                 62
food_package                         78
transport_package_mx                  0
sightseeing_package                   0
guided_tour_package                   0
insurance_package                   107
days_before_booked                  667
weather_at_arrival                 1593
tour_length                         153
special_requirements               3726
dtype: int64

In [149]:
df.special_requirements.fillna('none', inplace=True)
df.female_count.fillna(0, inplace=True)
df.male_count.fillna(0, inplace=True)
df.age_bracket.fillna('25-44', inplace=True)
df.first_time_visitor.fillna('Yes', inplace=True)
df.transport_package_international.fillna('No', inplace=True)

In [150]:
df.shape

(5852, 24)

In [151]:
df.isna().sum()

trip_ID                               0
visitor_nation                      126
age_bracket                           0
travelling_with                     338
female_count                          0
male_count                            0
key_activity                         65
trip_purpose                          0
first_time_visitor                    0
mainland_nights                       0
island_nights                         0
tour_arrangement                      0
transport_package_international       0
source_of_info                        0
package_accomodation                 62
food_package                         78
transport_package_mx                  0
sightseeing_package                   0
guided_tour_package                   0
insurance_package                   107
days_before_booked                  667
weather_at_arrival                 1593
tour_length                         153
special_requirements                  0
dtype: int64

In [152]:
df['total_travellers'] = df['female_count'] + df['male_count'] 

In [153]:
df.loc[(df['travelling_with'].isna()) & (df['total_travellers'] == 1), 'travelling_with'] = 'Alone'
df.loc[(df['travelling_with'].isna()) & (df['total_travellers'] == 2), 'travelling_with'] = 'With Spouse'

df.loc[(df.total_travellers == 0) & (df.travelling_with == 'With Spouse'), 'female_count'] = 1
df.loc[(df.total_travellers == 0) & (df.travelling_with.isin(['With Spouse', 'Alone'])), 'male_count'] = 1

df.loc[(df.total_travellers == 0) & (df.travelling_with.isin(['With Spouse and Children', 'With Other Friends/Relatives'])), 'female_count'] = 2
df.loc[(df.total_travellers == 0) & (df.travelling_with.isin(['With Spouse and Children', 'With Other Friends/Relatives'])), 'male_count'] = 2

In [154]:
df.loc[df['travelling_with'].isna(), 'travelling_with'] = 'Alone'



In [155]:
df.isna().sum()

trip_ID                               0
visitor_nation                      126
age_bracket                           0
travelling_with                       0
female_count                          0
male_count                            0
key_activity                         65
trip_purpose                          0
first_time_visitor                    0
mainland_nights                       0
island_nights                         0
tour_arrangement                      0
transport_package_international       0
source_of_info                        0
package_accomodation                 62
food_package                         78
transport_package_mx                  0
sightseeing_package                   0
guided_tour_package                   0
insurance_package                   107
days_before_booked                  667
weather_at_arrival                 1593
tour_length                         153
special_requirements                  0
total_travellers                      0


In [156]:
df.replace("Widlife Tourism", "Wildlife Tourism", inplace=True)

In [157]:
spending = pd.read_csv("country_wise_spending.csv")

In [158]:
import numpy as np

def str_toupper(st):
    return st.upper()

spending.replace('..', np.nan, inplace=True)
spending['Country Name'] = spending['Country Name'].map(str_toupper)
spending['2019 [YR2019]'].fillna(spending['2018 [YR2018]'], inplace=True)

In [159]:
df.shape

(5852, 25)

In [160]:
spending = spending[['Country Name', '2019 [YR2019]']]
spending.columns = ['visitor_nation', 'spending']

df = pd.merge(df, spending, on='visitor_nation', how='left')

In [161]:
df.shape

(5852, 26)

In [162]:
df[df['spending'].isna()]['visitor_nation'].unique()

array(['CONGO', 'SWIZERLAND', 'SPAIN', 'UNITED STATES OF AMERICA', 'UAE',
       'KENYA', 'RUSSIA', 'UNITED KINGDOM', 'CANADA', 'BURUNDI', 'DRC',
       'SWEDEN', 'ERITREA', nan, 'SWAZILAND', 'KOREA', 'TRINIDAD TOBACCO',
       'UNITED ARAB EMIRATES', 'SAUD ARABIA', 'CHINA', 'LATVIA',
       'VIETNAM', 'IRAN', 'SLOVAKIA', 'MACEDONIA', 'SCOTLAND', 'BARBADOS',
       'GAMBIA', 'COSTARICA', 'LITHUANIA', 'EGYPT', 'CZECH REPUBLIC',
       'PHILIPINES', 'BOSNIA', 'VENEZUELA', 'TURKEY', 'UKRAIN', 'COMORO',
       'MORROCO', 'YEMEN', 'TAIWAN', 'SOMALI', 'CAPE VERDE', 'ICELAND',
       'BURGARIA', 'MALT'], dtype=object)

In [163]:
df['spending'] = pd.to_numeric(df['spending'], errors='coerce')
df['spending'].describe()

count    3094.000000
mean        6.646242
std         2.794376
min         0.157818
25%         5.646109
50%         6.317842
75%         6.664603
max        26.551575
Name: spending, dtype: float64

In [ ]:
high_income_countries = [
    'BARBADOS', 'BURGARIA', 'CANADA', 'CZECH REPUBLIC', 'ICELAND', 'KOREA', 'LATVIA', 'LITHUANIA', 
    'MALT', 'RUSSIA', 'SAUD ARABIA', 'SCOTLAND', 'SLOVAKIA', 'SPAIN', 'SWEDEN', 'SWIZERLAND', 'TAIWAN', 
    'TRINIDAD TOBACCO', 'UAE', 'UNITED ARAB EMIRATES', 'UNITED KINGDOM', 'UNITED STATES OF AMERICA'
]

upper_middle_income_countries = ['BOSNIA', 'CHINA', 'COSTARICA', 'IRAN', 'TURKEY', 'UKRAIN']

lower_middle_income_countries = [
    'CAPE VERDE', 'COMORO', 'EGYPT', 'KENYA', 'MACEDONIA', 
    'MORROCO', 'PHILIPINES', 'SWAZILAND', 'VENEZUELA', 'VIETNAM'
]

low_income_countries = ['BURUNDI', 'CONGO', 'DRC', 'ERITREA', 'GAMBIA', 'SOMALI', 'YEMEN']

df.loc[df['visitor_nation'].isin(high_income_countries), 'spending'] = 10.49669737
df.loc[df['visitor_nation'].isin(upper_middle_income_countries), 'spending'] = 8.356929506
df.loc[df['visitor_nation'].isin(lower_middle_income_countries), 'spending'] = 7.781739015
df.loc[df['visitor_nation'].isin(low_income_countries), 'spending'] = 4.263060223


In [165]:
df.shape

(5852, 26)

In [166]:
df.loc[df['spending'].isna(), 'spending'] = df['spending'].median()

In [ ]:
# gdp was not used
# gdp = pd.read_csv('GDP.csv')

In [168]:
# gdp['Country '] = gdp['Country '].map(lambda x : x.upper())

In [169]:
# gdp = gdp[['Country ', '2018']]
# gdp.loc[gdp['Country '] == 'BERMUDA', '2018'] = 111540

In [170]:
# gdp.columns = ['country_name', 'gdp']

In [171]:
# gdp_map = gdp.set_index('country_name').to_dict('dict')['gdp']
# df['gdp'] = df['visitor_nation'].map(gdp_map)

In [172]:
df.shape

(5852, 26)

In [173]:
# df[df.gdp.isna()]

In [174]:
# df.loc[df.gdp.isna(), "gdp"] = 580

In [175]:
df.shape

(5852, 26)

In [176]:
df.fillna('unknown', inplace=True)

In [177]:

# decided based on the spending distribution seen above

# Using midpoints of the range
age_bracket_ordinal_mapping = {'<18':9, '18-24':21, '25-44':35, '45-64':55, '65+':70}

travelling_with_ordinal_mapping = {
    'Alone': 1,
    'With Spouse': 2,
    'With Other Friends/Relatives': 3,
    'With Children': 3,
    'With Spouse and Children': 3,
    'unknown': 3
} 

key_activity_ordinal_mapping = {
    "Wildlife Tourism": 1,
    "Conference Tourism": 2,
    "Business": 2,
    "Beach Tourism": 3,
    "Mountain Climbing": 4,
    "Cultural Tourism": 5,
    "Bird Tourism": 5,
    "Hunting Tourism": 6,
    "Diving and Sport Fishing": 4,
    "Other": 4,
    "unknown": 4 # give a neutral medium value
}

trip_purpose_mapping = {
    "Business": 1,                  
    "Meetings and Conference": 2,
    "Scientific and Academic": 2,
    "Volunteering": 3,
    "Visiting Friends and Relatives": 4,  
    "Leisure and Holidays": 5,
    "Other": 3,
    "Medical": 5,
    "unknown": 3
}


transport_package_international_ordinal_mapping = {'No':0, 'Yes':1}

first_time_visitor_ordinal_mapping = {'No':0, 'Yes':1}

tour_arrangement_ordinal_mapping = {'Independent':0, 'Package Tour':1}

transport_package_international_ordinal_mapping = {'No':0, 'Yes':1}

source_of_info_mapping = {
    "Inflight magazines": 1,
    "Trade fair": 1,
    "Mexican Mission Abroad": 2,
    "Others": 3,
    "Radio, TV, Web": 4,
    "Friends, relatives": 4,
    "Newspaper, magazines, brochures": 5,
    "Travel agent, tour operator": 5,      
    "unknown": 3
}


package_accomodation_ordinal_mapping = {'No':0, 'unknown': 1, 'Yes':2}

food_package_ordinal_mapping = {'No':0, 'unknown': 1, 'Yes':2}

transport_package_mx_ordinal_mapping = {'No':0, 'Yes':1}

sightseeing_package_ordinal_mapping = {'No':0, 'Yes':1}

guided_tour_package_ordinal_mapping = {'No':0, 'Yes':1}

insurance_package_ordinal_mapping = {'No':0, 'unknown': 1, 'Yes':2}


days_before_booked_ordinal_mapping = {
    "31-60": 45,
    "8-14": 11,
    "61-90": 75,
    "15-30 ": 22,
    "90+": 100,
    "1-7 ": 4,
    "unknown": 22 
}


weather_at_arrival_ordinal_mapping = {
     "cloudy,": 1,
    "Windy,": 2,
    "humid": 2,
    "sunny,": 2,
    "other": 2,
    "Rainy": 3,
    "Stormy": 3,
    "unknown": 2
}


tour_length_ordinal_mapping = {
    "7-14": 10,
    "1-6": 3,
    "30+": 40,
    "15-30": 22,
    "unknown": 20
}

special_requirements_ordinal_mapping = {
    "none": 1,
    "wheelchair,": 2,
    "dietary needs,": 2,
    "translator required.": 2    
}

first_time_visitor_ordinal_mapping = {'No':0, 'Yes':1}

In [178]:
df['age_bracket'] = df['age_bracket'].map(age_bracket_ordinal_mapping)
df['travelling_with'] = df['travelling_with'].map(travelling_with_ordinal_mapping)
df['key_activity'] = df['key_activity'].map(key_activity_ordinal_mapping)
df['trip_purpose'] = df['trip_purpose'].map(trip_purpose_mapping)
df['first_time_visitor'] = df['first_time_visitor'].map(first_time_visitor_ordinal_mapping)
df['tour_arrangement'] = df['tour_arrangement'].map(tour_arrangement_ordinal_mapping)
df['transport_package_international'] = df['transport_package_international'].map(transport_package_international_ordinal_mapping)
df['source_of_info'] = df['source_of_info'].map(source_of_info_mapping)
df['package_accomodation'] = df['package_accomodation'].map(package_accomodation_ordinal_mapping)
df['food_package'] = df['food_package'].map(food_package_ordinal_mapping)
df['transport_package_mx'] = df['transport_package_mx'].map(transport_package_mx_ordinal_mapping)
df['sightseeing_package'] = df['sightseeing_package'].map(sightseeing_package_ordinal_mapping)
df['guided_tour_package'] = df['guided_tour_package'].map(guided_tour_package_ordinal_mapping)
df['insurance_package'] = df['insurance_package'].map(insurance_package_ordinal_mapping)
df['days_before_booked'] = df['days_before_booked'].map(days_before_booked_ordinal_mapping)
df['weather_at_arrival'] = df['weather_at_arrival'].map(weather_at_arrival_ordinal_mapping)
df['tour_length'] = df['tour_length'].map(tour_length_ordinal_mapping)
df['special_requirements'] = df['special_requirements'].map(special_requirements_ordinal_mapping)

In [179]:
df['total_nights'] = df['mainland_nights'] + df['island_nights']
df['booked_late'] = df['days_before_booked'].map(lambda x: int(x == '1-7 '))
df['ratio'] = df['male_count']/df['female_count']
df['total_travellers'] = df['male_count'] + df['female_count']

In [180]:
df.shape

(5852, 29)

In [181]:
import numpy as np
df['ratio'].fillna(12, inplace=True)
df['ratio'].replace(np.inf, 12, inplace=True)

In [182]:
df.head(10)

,trip_ID,visitor_nation,age_bracket,travelling_with,female_count,male_count,key_activity,trip_purpose,first_time_visitor,mainland_nights,...,insurance_package,days_before_booked,weather_at_arrival,tour_length,special_requirements,total_travellers,spending,total_nights,booked_late,ratio
0,tour_id8gzpck76,CONGO,35,1,0.0,1.0,1,1,0,14,...,0,22,2,22,2,1.0,4.263060,14,0,12.0
1,tour_idow1zxkou,SWIZERLAND,55,2,1.0,1.0,1,5,1,8,...,0,75,3,3,1,2.0,10.496697,13,0,1.0
2,tour_idue7esfqz,MEXICO,55,3,2.0,0.0,5,5,1,3,...,0,4,3,40,1,2.0,2.414758,9,0,0.0
3,tour_idnj3mjzpb,JAPAN,35,3,1.0,1.0,1,5,1,5,...,0,100,2,10,1,2.0,3.191467,5,0,1.0
4,tour_ida3us5yk2,SPAIN,35,3,2.0,0.0,1,5,1,0,...,2,75,3,40,2,2.0,10.496697,6,0,0.0
5,tour_id2qpfclyc,ZIMBABWE,35,1,0.0,1.0,5,1,1,3,...,0,45,3,3,1,1.0,4.760583,3,0,12.0
6,tour_idope4voz1,ARGENTINA,35,3,2.0,2.0,3,5,1,4,...,0,75,1,3,1,4.0,14.788070,11,0,1.0
7,tour_iddyjku73v,ITALY,55,2,1.0,1.0,1,4,0,11,...,0,100,2,10,1,2.0,6.664603,11,0,1.0
8,tour_idlu2iemeg,UNITED STATES OF AMERICA,70,2,1.0,1.0,1,5,1,12,...,2,11,2,40,2,2.0,10.496697,12,0,1.0
9,tour_idw5htzhxs,ZIMBABWE,35,1,0.0,1.0,3,1,0,3,...,0,100,2,10,1,1.0,4.760583,3,0,12.0


In [184]:
df.isna().sum()

trip_ID                            0
visitor_nation                     0
age_bracket                        0
travelling_with                    0
female_count                       0
male_count                         0
key_activity                       0
trip_purpose                       0
first_time_visitor                 0
mainland_nights                    0
island_nights                      0
tour_arrangement                   0
transport_package_international    0
source_of_info                     0
package_accomodation               0
food_package                       0
transport_package_mx               0
sightseeing_package                0
guided_tour_package                0
insurance_package                  0
days_before_booked                 0
weather_at_arrival                 0
tour_length                        0
special_requirements               0
total_travellers                   0
spending                           0
total_nights                       0
b

In [186]:
df.shape

(5852, 29)

In [187]:
df.columns

Index(['trip_ID', 'visitor_nation', 'age_bracket', 'travelling_with',
       'female_count', 'male_count', 'key_activity', 'trip_purpose',
       'first_time_visitor', 'mainland_nights', 'island_nights',
       'tour_arrangement', 'transport_package_international', 'source_of_info',
       'package_accomodation', 'food_package', 'transport_package_mx',
       'sightseeing_package', 'guided_tour_package', 'insurance_package',
       'days_before_booked', 'weather_at_arrival', 'tour_length',
       'special_requirements', 'total_travellers', 'spending', 'total_nights',
       'booked_late', 'ratio'],
      dtype='object')

In [188]:
features = ['age_bracket', 'travelling_with',
       'female_count', 'male_count', 'key_activity', 'trip_purpose',
       'first_time_visitor', 'mainland_nights', 'island_nights',
       'tour_arrangement', 'transport_package_international', 'source_of_info',
       'package_accomodation', 'food_package', 'transport_package_mx',
       'sightseeing_package', 'guided_tour_package', 'insurance_package',
       'days_before_booked', 'weather_at_arrival', 'tour_length',
       'special_requirements','total_travellers', 'spending',
       'total_nights', 'booked_late', 'ratio']

In [189]:
df.to_csv("test_preprocessed.csv", index=False)

In [190]:
df.shape

(5852, 29)